In [35]:
# imports
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob

In [30]:
df_berlin = pd.read_csv('berlin_nextbike_final.csv')
df_berlin

,id,name,latitude,longitude,free_bikes,empty_slots,total_bikes
0,0007f94b88fd0f810e3f5435363bb0de,virtuell - S Treptower Park (Puschkinallee/Els...,52.492780,13.460247,0,5,5
1,000c1394c49338dd136955cfa83f0f02,U Ullsteinstraße (Staubenrauchbrücke),52.453146,13.384738,1,5,6
2,000d6738d33fd09713f9ba3d645eb083,Olivaer Platz (Nordwest),52.499709,13.313215,2,1,3
3,00262119a775f009135c009c642de971,Katzbachstraße,52.487904,13.376718,0,15,15
4,0029bd4fba89c502af0c1732591080cd,Greifswalder Straße/Lehderstraße,52.547140,13.447640,0,4,4
...,...,...,...,...,...,...,...
2710,ff67e2dfdad32ab14869cf1b5e094cbc,EDEKA Schnelle (Alt-Stralau),52.496545,13.466883,0,4,4
2711,ff688d1e5051c075ba93f4d29fef3b3e,Gotthardstraße/Holländerstraße,52.565208,13.335815,0,4,4
2712,ff779ad1b2291e55ce1ff346cbef2323,Ostendstraße,52.456006,13.509734,0,4,4
2713,ffa2ed58ad58a3e6648221924b929e70,Gerichtstraße/Adolfstraße,52.544916,13.367097,1,3,4


In [31]:
df_berlin[['latitude','longitude']] = df_berlin[['latitude','longitude']].apply(lambda x:round(x,4))
df_berlin.drop(columns='id')

,name,latitude,longitude,free_bikes,empty_slots,total_bikes
0,virtuell - S Treptower Park (Puschkinallee/Els...,52.4928,13.4602,0,5,5
1,U Ullsteinstraße (Staubenrauchbrücke),52.4531,13.3847,1,5,6
2,Olivaer Platz (Nordwest),52.4997,13.3132,2,1,3
3,Katzbachstraße,52.4879,13.3767,0,15,15
4,Greifswalder Straße/Lehderstraße,52.5471,13.4476,0,4,4
...,...,...,...,...,...,...
2710,EDEKA Schnelle (Alt-Stralau),52.4965,13.4669,0,4,4
2711,Gotthardstraße/Holländerstraße,52.5652,13.3358,0,4,4
2712,Ostendstraße,52.4560,13.5097,0,4,4
2713,Gerichtstraße/Adolfstraße,52.5449,13.3671,1,3,4


In [32]:
location_df = pd.DataFrame()
location_df['name'] = df_berlin['name']
location_df['latitude'] = df_berlin['latitude']
location_df['longitude'] = df_berlin['longitude']
location_df['latlon'] = location_df['latitude'].astype(str) + "," + location_df['longitude'].astype(str)
location_df

,name,latitude,longitude,latlon
0,virtuell - S Treptower Park (Puschkinallee/Els...,52.4928,13.4602,"52.4928,13.4602"
1,U Ullsteinstraße (Staubenrauchbrücke),52.4531,13.3847,"52.4531,13.3847"
2,Olivaer Platz (Nordwest),52.4997,13.3132,"52.4997,13.3132"
3,Katzbachstraße,52.4879,13.3767,"52.4879,13.3767"
4,Greifswalder Straße/Lehderstraße,52.5471,13.4476,"52.5471,13.4476"
...,...,...,...,...
2710,EDEKA Schnelle (Alt-Stralau),52.4965,13.4669,"52.4965,13.4669"
2711,Gotthardstraße/Holländerstraße,52.5652,13.3358,"52.5652,13.3358"
2712,Ostendstraße,52.4560,13.5097,"52.456,13.5097"
2713,Gerichtstraße/Adolfstraße,52.5449,13.3671,"52.5449,13.3671"


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [33]:
for idx, n in enumerate(location_df['name']):
    latlon = location_df['latlon'][idx]
    foursquare_url = f"https://places-api.foursquare.com/places/search?ll={latlon}&radius=1000"
    foursquare_key = os.environ['foursquare']

    headers = {
    "accept": "application/json",
    "X-Places-Api-Version": "2025-06-17",
    "authorization": f"Bearer {foursquare_key}"
    }

    response = requests.get(foursquare_url, headers=headers)
    foursquare_data = response.json()
    filename = f'fs_data{idx}.json'
    path = 'C:/Users/vlouc/OneDrive/Documents/GitHub/LHL-Stats-Project/data/foursquare_data'
    fullpath = os.path.join(path, filename)
    #foursquare_df = pd.json_normalize(foursquare_data['results'])
    with open(fullpath, 'w') as f:
        json.dump(foursquare_data, f, ensure_ascii=False)


In [36]:
file_paths = glob.glob('C:/Users/vlouc/OneDrive/Documents/GitHub/LHL-Stats-Project/data/foursquare_data/*.json')

with open('output-file.json', 'w') as output_file:
    for file_path in file_paths:
        with open(file_path, 'r') as input_file:
            output_file.write(input_file.read() + '\n')

In [37]:
with open('output-file.json', 'r') as f:
    data = [json.loads(line) for line in f]

In [38]:
results = pd.json_normalize(data, record_path='results')
results.to_csv('results.csv')

In [39]:
results

,fsq_place_id,latitude,longitude,categories,date_created,date_refreshed,distance,link,name,placemaker_url,...,social_media.twitter,related_places.parent.fsq_place_id,related_places.parent.categories,related_places.parent.name,social_media.facebook_id,related_places.children,chains,email,social_media.instagram,store_id
0,4c334540213c2d7fbaa0365d,52.491981,13.459416,[{'fsq_category_id': '4eb1c0253b7b52c0e1adc2e9...,2010-07-06,2025-07-23,105,/places/4c334540213c2d7fbaa0365d,Der Holländer,https://foursquare.com/placemakers/review-plac...,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,518b635e498eee9194015820,52.492884,13.460402,[{'fsq_category_id': '4bf58dd8d48988d16a941735...,2013-05-09,2025-07-13,16,/places/518b635e498eee9194015820,Café Bäckerei Sesam,https://foursquare.com/placemakers/review-plac...,...,,4b24d167f964a520fa6924e3,[{'fsq_category_id': '4bf58dd8d48988d1fc931735...,S Treptower Park,322392024556080,NaN,NaN,NaN,NaN,NaN
2,4f953548e4b03828ff00588f,52.492191,13.458811,[{'fsq_category_id': '4bf58dd8d48988d110941735...,2012-04-23,2025-07-25,115,/places/4f953548e4b03828ff00588f,Da Gelo,https://foursquare.com/placemakers/review-plac...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51c6f175498e16a8c1e01a66,52.495205,13.462637,[{'fsq_category_id': '4bf58dd8d48988d11f941735...,2013-06-23,2025-07-26,313,/places/51c6f175498e16a8c1e01a66,Else,https://foursquare.com/placemakers/review-plac...,...,,NaN,NaN,NaN,388365811281424,"[{'fsq_place_id': '5d696def50044000089894df', ...",NaN,NaN,NaN,NaN
4,512f1ffae4b0aa2e908ed3f8,52.493718,13.459795,[{'fsq_category_id': '4bf58dd8d48988d16a941735...,2013-02-28,2025-04-15,105,/places/512f1ffae4b0aa2e908ed3f8,BackWerk,https://foursquare.com/placemakers/review-plac...,...,_backwerk_,NaN,NaN,NaN,NaN,NaN,[{'fsq_chain_id': '556e5dcdbd6a82902e29a781'}],info@back-werk.com,meinbackwerk,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27157,4bd03152b221c9b6bd31d3d0,52.497416,13.295766,[{'fsq_category_id': '4bf58dd8d48988d1c0941735...,2010-04-22,2025-05-27,146,/places/4bd03152b221c9b6bd31d3d0,Tre Terre,https://foursquare.com/placemakers/review-plac...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,treterre@gmx.de,NaN,NaN
27158,528b8f70498ec85f24e2c6e5,52.497997,13.297844,[{'fsq_category_id': '5745c2e4498e11e7bccabdbd...,2013-11-19,2025-07-24,255,/places/528b8f70498ec85f24e2c6e5,Rossmann,https://foursquare.com/placemakers/review-plac...,...,rossmann,NaN,NaN,NaN,354412263434,NaN,"[{'fsq_chain_id': '556e1846a7c82e6b72513d5f', ...",service@rossmann.de,mein_rossmann,2876
27159,4d39bbff1fc36dcbdd755ef5,52.496737,13.292919,[{'fsq_category_id': '4bf58dd8d48988d110941735...,2011-01-21,2025-07-25,201,/places/4d39bbff1fc36dcbdd755ef5,De Santi,https://foursquare.com/placemakers/review-plac...,...,NaN,NaN,NaN,NaN,108458395882871,NaN,NaN,desanti45@gmx.de,NaN,NaN
27160,4bdc12c53904a593f4c44c9e,52.497427,13.293464,[{'fsq_category_id': '4bf58dd8d48988d113941735...,2010-05-01,2025-06-04,211,/places/4bdc12c53904a593f4c44c9e,MO:IM,https://foursquare.com/placemakers/review-plac...,...,,NaN,NaN,NaN,NaN,NaN,NaN,leelsw.119@hanmail.net,berlin.moim,NaN


In [40]:
results['categories'] = results['categories'].apply(lambda lst: [d['name'] for d in lst])

In [41]:
results_exploded = results.explode('categories')
results_exploded = results_exploded.rename(columns={'categories': 'category_name'})

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [42]:
results_exploded = results_exploded[results_exploded['category_name'].notnull()]

In [43]:
results_exploded = results_exploded[results_exploded['category_name'].str.contains("Restaurant|Bar|Cafe|Caf|Museum|Gallery|Monastery|Church|Retail|Brewery|Winery|Bakery|Pub|Wine|Beer|Coffee")]

In [44]:
results_exploded

,fsq_place_id,latitude,longitude,category_name,date_created,date_refreshed,distance,link,name,placemaker_url,...,social_media.twitter,related_places.parent.fsq_place_id,related_places.parent.categories,related_places.parent.name,social_media.facebook_id,related_places.children,chains,email,social_media.instagram,store_id
1,518b635e498eee9194015820,52.492884,13.460402,Bakery,2013-05-09,2025-07-13,16,/places/518b635e498eee9194015820,Café Bäckerei Sesam,https://foursquare.com/placemakers/review-plac...,...,,4b24d167f964a520fa6924e3,[{'fsq_category_id': '4bf58dd8d48988d1fc931735...,S Treptower Park,322392024556080,NaN,NaN,NaN,NaN,NaN
2,4f953548e4b03828ff00588f,52.492191,13.458811,Italian Restaurant,2012-04-23,2025-07-25,115,/places/4f953548e4b03828ff00588f,Da Gelo,https://foursquare.com/placemakers/review-plac...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,512f1ffae4b0aa2e908ed3f8,52.493718,13.459795,Bakery,2013-02-28,2025-04-15,105,/places/512f1ffae4b0aa2e908ed3f8,BackWerk,https://foursquare.com/placemakers/review-plac...,...,_backwerk_,NaN,NaN,NaN,NaN,NaN,[{'fsq_chain_id': '556e5dcdbd6a82902e29a781'}],info@back-werk.com,meinbackwerk,NaN
6,4e0486ffd1649538d96d0821,52.493991,13.463445,Seafood Restaurant,2011-06-24,2025-06-27,256,/places/4e0486ffd1649538d96d0821,Hafenräucherei Treptow „Löcknitz“,https://foursquare.com/placemakers/review-plac...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4c31b09f16adc928d1f5c09c,52.492109,13.458576,Chinese Restaurant,2010-07-05,2022-02-19,134,/places/4c31b09f16adc928d1f5c09c,Asia Palast im Cinestar,https://foursquare.com/placemakers/review-plac...,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27157,4bd03152b221c9b6bd31d3d0,52.497416,13.295766,Mediterranean Restaurant,2010-04-22,2025-05-27,146,/places/4bd03152b221c9b6bd31d3d0,Tre Terre,https://foursquare.com/placemakers/review-plac...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,treterre@gmx.de,NaN,NaN
27159,4d39bbff1fc36dcbdd755ef5,52.496737,13.292919,Italian Restaurant,2011-01-21,2025-07-25,201,/places/4d39bbff1fc36dcbdd755ef5,De Santi,https://foursquare.com/placemakers/review-plac...,...,NaN,NaN,NaN,NaN,108458395882871,NaN,NaN,desanti45@gmx.de,NaN,NaN
27160,4bdc12c53904a593f4c44c9e,52.497427,13.293464,Korean Restaurant,2010-05-01,2025-06-04,211,/places/4bdc12c53904a593f4c44c9e,MO:IM,https://foursquare.com/placemakers/review-plac...,...,,NaN,NaN,NaN,NaN,NaN,NaN,leelsw.119@hanmail.net,berlin.moim,NaN
27161,4b280a86f964a520178e24e3,52.498475,13.296616,German Restaurant,2009-12-15,2025-07-25,271,/places/4b280a86f964a520178e24e3,Haus der 100 Biere,https://foursquare.com/placemakers/review-plac...,...,,NaN,NaN,NaN,161691123854266,NaN,NaN,mail@mommseneck.de,NaN,NaN


Put your parsed results into a DataFrame

In [46]:
berlin_df = pd.DataFrame()
berlin_df['fsq_place_id'] = results_exploded['fsq_place_id']
berlin_df['name'] = results_exploded['name']
berlin_df['category_name'] = results_exploded['category_name']
berlin_df['address'] = results_exploded['location.address']
berlin_df['postcode'] = results_exploded['location.postcode']
berlin_df['latitude'] = results_exploded['latitude']
berlin_df['longitude'] = results_exploded['longitude']
berlin_df['distance'] = results_exploded['distance']
berlin_df.to_csv('fsq_berlin_df.csv')
berlin_df

,fsq_place_id,name,category_name,address,postcode,latitude,longitude,distance
1,518b635e498eee9194015820,Café Bäckerei Sesam,Bakery,Puschkinallee 14 a,12435,52.492884,13.460402,16
2,4f953548e4b03828ff00588f,Da Gelo,Italian Restaurant,Elsenstr. 115,12435,52.492191,13.458811,115
4,512f1ffae4b0aa2e908ed3f8,BackWerk,Bakery,Elsenstraße 2,12435,52.493718,13.459795,105
6,4e0486ffd1649538d96d0821,Hafenräucherei Treptow „Löcknitz“,Seafood Restaurant,Hafen Treptow,12435,52.493991,13.463445,256
8,4c31b09f16adc928d1f5c09c,Asia Palast im Cinestar,Chinese Restaurant,Am Treptower Park,12435,52.492109,13.458576,134
...,...,...,...,...,...,...,...,...
27157,4bd03152b221c9b6bd31d3d0,Tre Terre,Mediterranean Restaurant,Joachim-Friedrich-Str. 45,10711,52.497416,13.295766,146
27159,4d39bbff1fc36dcbdd755ef5,De Santi,Italian Restaurant,Westfälische Str. 45,10711,52.496737,13.292919,201
27160,4bdc12c53904a593f4c44c9e,MO:IM,Korean Restaurant,Kurfürstendamm 134,10711,52.497427,13.293464,211
27161,4b280a86f964a520178e24e3,Haus der 100 Biere,German Restaurant,Kurfürstendamm 100,10709,52.498475,13.296616,271


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
## I was unable to access the Yelp API so I didn't complete this part of the exercise.

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating